In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install unsloth-zoo
!pip install kagglehub
!pip install tqdm

In [ ]:
import kagglehub
import os
import json
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import TextStreamer
from datasets import load_dataset
from tqdm import tqdm

finetune_1_prime_nos_path = kagglehub.dataset_download('raymanodeep/finetune-1-prime-nos')
str(finetune_1_prime_nos_path)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


'/root/.cache/kagglehub/datasets/raymanodeep/finetune-1-prime-nos/versions/1'

In [ ]:
from tqdm import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

drive_path = "/content/drive/MyDrive"
os.mkdir(path = os.path.join(drive_path ,"finetuning"))

Mounted at /content/drive


In [ ]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
torch.device(device=device)

device(type='cuda')

## For LLAMA-3-8b 4-bit
 > for regular dataset




In [ ]:
import json
qa_data = []
qa_test_data = []

with open(str(finetune_1_prime_nos_path) +"/replaced_dataset/test_output.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        qa_test_data.append(json.loads(line.strip()))

with open(str(finetune_1_prime_nos_path) +"/replaced_dataset/train_output.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        qa_data.append(json.loads(line.strip()))

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='unsloth/llama-3-8b-bnb-4bit', vocab_size=128000, model_max_length=8192, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'pad_token': '<|reserved_special_token_250|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|reserved_special_token_2|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128

In [ ]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

EOS_TOKEN = tokenizer.eos_token
llama3_8b_test_answers = []

for i in tqdm(range(len(qa_test_data))):

  item = qa_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_8b_test_answers.append(response)


  '''from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

EOS_TOKEN = tokenizer.eos_token
llama_1b_test_answers = []
llama_1b_finetuned_test_answers = []

for item in qa_test_data:
  prompt = "solve the following grade school math problem like this question:Eliza's rate per hour for the first 41 hours she works each week is $10.49. She also receives an overtime pay of 3.3 times her regular hourly rate. If Eliza worked for 47 hours this week, how much are her earnings for this week? answer : Eliza is entitled to 47 -41 = <<47-41=6.0>>6.0 hours overtime pay.\nHer hourly rate for the overtime pay is $10.49 x 3.3 = $<<10.49*3.3=34.62>>34.62.\nSo, Eliza will receive $34.62 x 6.0 =$<<34.62*6.0=207.7>>207.7 for overtime pay.\nHer regular weekly earning is $10.49 x 41 = $<<10.49*41=430.09>>430.09.\nThus, Eliza will receive a total of $430.09 + $207.7 = $<<430.09+207.7=637.79>>637.79 for this week's work.\n#### 637.79 \n question : "+item['question'] +" answer: "+ EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama_1b_test_answers.append(response)'''

<|begin_of_text|>solve the following grade school math problem question : Janet’s ducks lay 17 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2.2800000000000002 per fresh duck egg. How much in dollars does she make every day at the farmers' market?<|end_of_text|><|begin_of_text|>://<|end_of_text|>
<|begin_of_text|>solve the following grade school math problem question : A robe takes 3 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?<|end_of_text|><|begin_of_text|>://
solve the following grade school math problem question : A robe takes 3 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?
A robe takes 3 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?
3. 2  1. 4  2. 6  3. 8<|end_of_text|>
<|begin_of_text|>solve the following grade school math p

KeyboardInterrupt: 

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
chat_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instruction = "solve the following grade school math problem"
    inputs       = examples["question"]
    outputs      = examples["answer"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = chat_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

# Uncomment for the correct dataset
dataset = load_dataset('json', data_files=str(finetune_1_prime_nos_path) +"/replaced_dataset/train_output.jsonl", )# dataset = load_dataset("<your_hf_account>/med-qa", split = "train")

dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/7026 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'text'],
        num_rows: 7026
    })
})

In [ ]:
print(len(dataset["train"]))

7026


In [ ]:
print(dataset["train"]['text'][:5])

['\n### Instruction:\nsolve the following grade school math problem\n\n### Input:\nNatalia sold clips to 53 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\n\n### Response:\nNatalia sold 53/2 = <<53/2=26.5>>26.5 clips in May.\nNatalia sold 53+26.5 = <<53+26.5=79.5>>79.5 clips altogether in April and May.\n#### 79.5<|end_of_text|>', '\n### Instruction:\nsolve the following grade school math problem\n\n### Input:\nWeng earns $13.0 an hour for babysitting. Yesterday, she just did 53 minutes of babysitting. How much did she earn?\n\n### Response:\nWeng earns 13.0/60 = $<<13.0/60=0.22>>0.22 per minute.\nWorking 53 minutes, she earned 0.22 x 53 = $<<0.22*53=11.48>>11.48.\n#### 11.48<|end_of_text|>', "\n### Instruction:\nsolve the following grade school math problem\n\n### Input:\nBetty is saving money for a new wallet which costs $100.25. Betty has only half of the money she needs. Her parents decided to give 

In [ ]:
count_text = 0
count_question = 0
count_answer = 0

for item in dataset['train']:
  if item['text'] == None:
    count_text = count_text+ 1
  if item['question'] == None:
    count_question +=1
  if item['answer'] == None:
    count_question +=1

In [ ]:
if count_text == 0 and count_question == 0 and count_answer == 0:
  pass
else:
  0/0

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        num_train_epochs = 1
    ),
)

Map (num_proc=2):   0%|          | 0/7026 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#trainer_stats = trainer.train()

In [ ]:
'''#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")'''

'#@title Show final memory and time stats\nused_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)\nused_memory_for_lora = round(used_memory - start_gpu_memory, 3)\nused_percentage = round(used_memory         /max_memory*100, 3)\nlora_percentage = round(used_memory_for_lora/max_memory*100, 3)\nprint(f"{trainer_stats.metrics[\'train_runtime\']} seconds used for training.")\nprint(f"{round(trainer_stats.metrics[\'train_runtime\']/60, 2)} minutes used for training.")\nprint(f"Peak reserved memory = {used_memory} GB.")\nprint(f"Peak reserved memory for training = {used_memory_for_lora} GB.")\nprint(f"Peak reserved memory % of max memory = {used_percentage} %.")\nprint(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")'

In [ ]:
from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)


llama3_8b_finetuned_test_answers = []

for i in tqdm(range(len(qa_test_data))):

  item = qa_test_data[i]

  prompt = "solve the following grade school math problem question : "+item['question'] +" answer: "+ EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_8b_finetuned_test_answers.append(response)

  '''from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)


llama_1b_finetuned_test_answers = []


for item in qa_test_data:
  prompt = "solve the following grade school math problem like this question :Eliza's rate per hour for the first 41 hours she works each week is $10.49. She also receives an overtime pay of 3.3 times her regular hourly rate. If Eliza worked for 47 hours this week, how much are her earnings for this week? answer : Eliza is entitled to 47 -41 = <<47-41=6.0>>6.0 hours overtime pay.\nHer hourly rate for the overtime pay is $10.49 x 3.3 = $<<10.49*3.3=34.62>>34.62.\nSo, Eliza will receive $34.62 x 6.0 =$<<34.62*6.0=207.7>>207.7 for overtime pay.\nHer regular weekly earning is $10.49 x 41 = $<<10.49*41=430.09>>430.09.\nThus, Eliza will receive a total of $430.09 + $207.7 = $<<430.09+207.7=637.79>>637.79 for this week's work.\n#### 637.79 \n question : "+item['question'] +" answer: "+ EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama_1b_finetuned_test_answers.append(response)'''

<|begin_of_text|>solve the following grade school math problem question : Janet’s ducks lay 17 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2.2800000000000002 per fresh duck egg. How much in dollars does she make every day at the farmers' market? answer: <|end_of_text|><|begin_of_text|>:// 2.2800000000000002(17-6) = $2.2800000000000002(11) = $24.800000000000002
solve the following grade school math problem question : Janet’s ducks lay 17 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2.2800000000000002 per fresh duck egg. How much in dollars does she make every day at the farmers' market? answer: 2.280000000000000
<|begin_of_text|>solve the following grade school math problem question : A robe takes 3 bolts of blue fiber and half that much wh

KeyboardInterrupt: 

In [ ]:
llama3_8b_results = {}
llama3_8b_dir = os.path.join(drive_path ,"finetuning" , "llama3_8b")
llama3_8b_results["test_results"]=llama3_8b_test_answers
llama3_8b_results["finetuned_results"] = llama3_8b_finetuned_test_answers

llama3_8b_dir = os.path.join(drive_path ,"finetuning" , "llama3_8b")


with open( os.path.join(llama3_8b_dir,"llama3_8b_results.json"), "w") as outfile:
    json.dump(llama3_8b_results, outfile)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/finetuning/llama3_8b/llama3_8b_results.json'

In [ ]:
model.save_pretrained(os.path.join(llama3_8b_dir,"lora_model_llama3_8b")) # Local saving
tokenizer.save_pretrained(os.path.join(llama3_8b_dir,"lora_model_llama3_8b"))


# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('/content/drive/MyDrive/finetuning/llama3_8b/lora_model_llama3_8b/tokenizer_config.json',
 '/content/drive/MyDrive/finetuning/llama3_8b/lora_model_llama3_8b/special_tokens_map.json',
 '/content/drive/MyDrive/finetuning/llama3_8b/lora_model_llama3_8b/tokenizer.json')

In [ ]:
del llama3_8b_results
del llama3_8b_finetuned_test_answers
del llama3_8b_test_answers
del model
del tokenizer
del inputs
del outputs
del response
del text_streamer
del trainer
del count_text
del count_question
del count_answer
del dataset


## LLama-3-8b 4-bit

> for socratic dataset



In [ ]:
qa_socratic_data = []
qa_socratic_test_data =[]
with open(str(finetune_1_prime_nos_path) +"/replaced_dataset/test_socratic_output.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        qa_socratic_test_data.append(json.loads(line.strip()))


with open(str(finetune_1_prime_nos_path) +"/replaced_dataset/train_socratic_output.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        qa_socratic_data.append(json.loads(line.strip()))

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

EOS_TOKEN = tokenizer.eos_token
llama3_8b_test_socratic_answers = []


for i in tqdm(range(len(qa_socratic_test_data))):

  item = qa_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_8b_test_socratic_answers.append(response)


  '''from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

EOS_TOKEN = tokenizer.eos_token
llama_1b_test_answers = []
llama_1b_finetuned_test_answers = []

for item in qa_test_data:
  prompt = "solve the following grade school math problem like this question:Eliza's rate per hour for the first 41 hours she works each week is $10.49. She also receives an overtime pay of 3.3 times her regular hourly rate. If Eliza worked for 47 hours this week, how much are her earnings for this week? answer : Eliza is entitled to 47 -41 = <<47-41=6.0>>6.0 hours overtime pay.\nHer hourly rate for the overtime pay is $10.49 x 3.3 = $<<10.49*3.3=34.62>>34.62.\nSo, Eliza will receive $34.62 x 6.0 =$<<34.62*6.0=207.7>>207.7 for overtime pay.\nHer regular weekly earning is $10.49 x 41 = $<<10.49*41=430.09>>430.09.\nThus, Eliza will receive a total of $430.09 + $207.7 = $<<430.09+207.7=637.79>>637.79 for this week's work.\n#### 637.79 \n question : "+item['question'] +" answer: "+ EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama_1b_test_answers.append(response)'''

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


In [ ]:

chat_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instruction = "solve the following grade school math problem"
    inputs       = examples["question"]
    outputs      = examples["answer"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = chat_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

# Uncomment for the correct dataset
dataset = load_dataset('json', data_files=str(finetune_1_prime_nos_path) +"/replaced_dataset/train_socratic_output.jsonl", )# dataset = load_dataset("<your_hf_account>/med-qa", split = "train")

dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/7028 [00:00<?, ? examples/s]

In [ ]:
count_text = 0
count_question = 0
count_answer = 0

for item in dataset['train']:
  if item['text'] == None:
    count_text = count_text+ 1
  if item['question'] == None:
    count_question +=1
  if item['answer'] == None:
    count_question +=1

if count_text == 0 and count_question == 0 and count_answer == 0:
  pass
else:
  0/0

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        num_train_epochs = 1
    ),
)

Map (num_proc=2):   0%|          | 0/7028 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)


llama3_8b_finetuned_test_socratic_answers = []


for i in tqdm(range(len(qa_socratic_test_data))):

  item = qa_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] +" answer: "+ EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_8b_finetuned_test_socratic_answers.append(response)

  '''from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)


llama_1b_finetuned_test_answers = []


for item in qa_test_data:
  prompt = "solve the following grade school math problem like this question :Eliza's rate per hour for the first 41 hours she works each week is $10.49. She also receives an overtime pay of 3.3 times her regular hourly rate. If Eliza worked for 47 hours this week, how much are her earnings for this week? answer : Eliza is entitled to 47 -41 = <<47-41=6.0>>6.0 hours overtime pay.\nHer hourly rate for the overtime pay is $10.49 x 3.3 = $<<10.49*3.3=34.62>>34.62.\nSo, Eliza will receive $34.62 x 6.0 =$<<34.62*6.0=207.7>>207.7 for overtime pay.\nHer regular weekly earning is $10.49 x 41 = $<<10.49*41=430.09>>430.09.\nThus, Eliza will receive a total of $430.09 + $207.7 = $<<430.09+207.7=637.79>>637.79 for this week's work.\n#### 637.79 \n question : "+item['question'] +" answer: "+ EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama_1b_finetuned_test_answers.append(response)'''

  0%|          | 0/1211 [00:00<?, ?it/s]

<|begin_of_text|>solve the following grade school math problem question : Janet’s ducks lay 17 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2.5300000000000002 per fresh duck egg. How much in dollars does she make every day at the farmers' market? answer: <|end_of_text|><|begin_of_text|>://2.5300000000000002<|end_of_text|>


  0%|          | 1/1211 [00:01<29:22,  1.46s/it]

<|begin_of_text|>solve the following grade school math problem question : A robe takes 3 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take? answer: <|end_of_text|><|begin_of_text|>:// 6 bolts of blue fiber and 3 bolts of white fiber. I would like to know how to solve this question. I was told that I should add 3 and 1/2 and get 7/2. 

  0%|          | 1/1211 [00:07<2:30:18,  7.45s/it]


KeyboardInterrupt: 

In [ ]:
llama3_8b_socratic_results = {}
llama3_8b_dir = os.path.join(drive_path ,"finetuning" , "llama3_8b")

llama3_8b_socratic_results["test_results"]=llama3_8b_test_socratic_answers
llama3_8b_socratic_results["finetuned_results"] = llama3_8b_finetuned_test_socratic_answers



with open( os.path.join(llama3_8b_dir,"llama3_8b_socratic_results.json"), "w") as outfile:
    json.dump(llama3_8b_socratic_results, outfile)

NameError: name 'llama3_8b_test_socratic_answers' is not defined

In [ ]:
model.save_pretrained(os.path.join(llama3_8b_dir,"lora_model_socratic_llama3_8b")) # Local saving
tokenizer.save_pretrained(os.path.join(llama3_8b_dir,"lora_model_socratic_llama3_8b"))


# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('/content/drive/MyDrive/finetuning/llama3_8b/lora_model_socratic_llama3_8b/tokenizer_config.json',
 '/content/drive/MyDrive/finetuning/llama3_8b/lora_model_socratic_llama3_8b/special_tokens_map.json',
 '/content/drive/MyDrive/finetuning/llama3_8b/lora_model_socratic_llama3_8b/tokenizer.json')

In [ ]:
del llama3_8b_socratic_results
del llama3_8b_finetuned_test_socratic_answers
del llama3_8b_test_socratic_answers
del model
del tokenizer
del inputs
del outputs
del response
del text_streamer
del trainer
del count_text
del count_question
del count_answer
del dataset


NameError: name 'llama3_8b_test_socratic_answers' is not defined

## For LLama3 - 1b
>for regular dataset


In [ ]:
import json
qa_data = []
qa_test_data = []

with open(str(finetune_1_prime_nos_path) +"/replaced_dataset/test_output.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        qa_test_data.append(json.loads(line.strip()))

with open(str(finetune_1_prime_nos_path) +"/replaced_dataset/train_output.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        qa_data.append(json.loads(line.strip()))

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)




==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [ ]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

EOS_TOKEN = tokenizer.eos_token
llama3_1b_test_answers = []

for i in tqdm(range(len(qa_test_data))):

  item = qa_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_1b_test_answers.append(response)


  '''from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

EOS_TOKEN = tokenizer.eos_token
llama_1b_test_answers = []
llama_1b_finetuned_test_answers = []

for item in qa_test_data:
  prompt = "solve the following grade school math problem like this question:Eliza's rate per hour for the first 41 hours she works each week is $10.49. She also receives an overtime pay of 3.3 times her regular hourly rate. If Eliza worked for 47 hours this week, how much are her earnings for this week? answer : Eliza is entitled to 47 -41 = <<47-41=6.0>>6.0 hours overtime pay.\nHer hourly rate for the overtime pay is $10.49 x 3.3 = $<<10.49*3.3=34.62>>34.62.\nSo, Eliza will receive $34.62 x 6.0 =$<<34.62*6.0=207.7>>207.7 for overtime pay.\nHer regular weekly earning is $10.49 x 41 = $<<10.49*41=430.09>>430.09.\nThus, Eliza will receive a total of $430.09 + $207.7 = $<<430.09+207.7=637.79>>637.79 for this week's work.\n#### 637.79 \n question : "+item['question'] +" answer: "+ EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama_1b_test_answers.append(response)'''





In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)



In [ ]:
chat_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instruction = "solve the following grade school math problem"
    inputs       = examples["question"]
    outputs      = examples["answer"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = chat_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

# Uncomment for the correct dataset
dataset = load_dataset('json', data_files=str(finetune_1_prime_nos_path) +"/replaced_dataset/train_output.jsonl", )# dataset = load_dataset("<your_hf_account>/med-qa", split = "train")

dataset = dataset.map(formatting_prompts_func, batched = True,)



In [ ]:
count_text = 0
count_question = 0
count_answer = 0

for item in dataset['train']:
  if item['text'] == None:
    count_text = count_text+ 1
  if item['question'] == None:
    count_question +=1
  if item['answer'] == None:
    count_question +=1




In [ ]:
if count_text == 0 and count_question == 0 and count_answer == 0:
  pass
else:
  0/0


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        num_train_epochs = 1
    ),
)



In [ ]:
from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)


llama3_1b_finetuned_test_answers = []

for i in tqdm(range(len(qa_test_data))):

  item = qa_test_data[i]

  prompt = "solve the following grade school math problem question : "+item['question'] +" answer: "+ EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_1b_finetuned_test_answers.append(response)

  '''from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)


llama_1b_finetuned_test_answers = []


for item in qa_test_data:
  prompt = "solve the following grade school math problem like this question :Eliza's rate per hour for the first 41 hours she works each week is $10.49. She also receives an overtime pay of 3.3 times her regular hourly rate. If Eliza worked for 47 hours this week, how much are her earnings for this week? answer : Eliza is entitled to 47 -41 = <<47-41=6.0>>6.0 hours overtime pay.\nHer hourly rate for the overtime pay is $10.49 x 3.3 = $<<10.49*3.3=34.62>>34.62.\nSo, Eliza will receive $34.62 x 6.0 =$<<34.62*6.0=207.7>>207.7 for overtime pay.\nHer regular weekly earning is $10.49 x 41 = $<<10.49*41=430.09>>430.09.\nThus, Eliza will receive a total of $430.09 + $207.7 = $<<430.09+207.7=637.79>>637.79 for this week's work.\n#### 637.79 \n question : "+item['question'] +" answer: "+ EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama_1b_finetuned_test_answers.append(response)'''




In [ ]:
llama3_1b_results = {}
llama3_1b_dir = os.path.join(drive_path ,"finetuning" , "llama3_1b")

llama3_1b_results["test_results"]=llama3_1b_test_answers
llama3_1b_results["finetuned_results"] = llama3_1b_finetuned_test_answers



with open( os.path.join(llama3_1b_dir,"llama3_1b_results.json"), "w") as outfile:
    json.dump(llama3_1b_results, outfile)

model.save_pretrained(os.path.join(llama3_1b_dir,"lora_model_llama3_1b")) # Local saving
tokenizer.save_pretrained(os.path.join(llama3_1b_dir,"lora_model_llama3_1b"))


# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [ ]:
del llama3_1b_results
del llama3_1b_finetuned_test_answers
del llama3_1b_test_answers
del model
del tokenizer
del inputs
del outputs
del response
del text_streamer
del trainer
del count_text
del count_question
del count_answer
del dataset


## For LLama3-1b
> socratic dataset

In [ ]:
qa_socratic_data = []
qa_socratic_test_data =[]
with open(str(finetune_1_prime_nos_path) +"/replaced_dataset/test_socratic_output.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        qa_socratic_test_data.append(json.loads(line.strip()))


with open(str(finetune_1_prime_nos_path) +"/replaced_dataset/train_socratic_output.jsonl","r", encoding="utf-8") as f:
    for line in f:
        # Parse each line as a JSON object and append to the list
        qa_socratic_data.append(json.loads(line.strip()))

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)


from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

EOS_TOKEN = tokenizer.eos_token
llama3_1b_test_socratic_answers = []

for i in tqdm(range(len(qa_socratic_test_data))):

  item = qa_socratic_test_data[i]
  prompt = "solve the following grade school math problem question : "+item['question'] + EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_1b_test_socratic_answers.append(response)


  '''from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

EOS_TOKEN = tokenizer.eos_token
llama_1b_test_answers = []
llama_1b_finetuned_test_answers = []

for item in qa_test_data:
  prompt = "solve the following grade school math problem like this question:Eliza's rate per hour for the first 41 hours she works each week is $10.49. She also receives an overtime pay of 3.3 times her regular hourly rate. If Eliza worked for 47 hours this week, how much are her earnings for this week? answer : Eliza is entitled to 47 -41 = <<47-41=6.0>>6.0 hours overtime pay.\nHer hourly rate for the overtime pay is $10.49 x 3.3 = $<<10.49*3.3=34.62>>34.62.\nSo, Eliza will receive $34.62 x 6.0 =$<<34.62*6.0=207.7>>207.7 for overtime pay.\nHer regular weekly earning is $10.49 x 41 = $<<10.49*41=430.09>>430.09.\nThus, Eliza will receive a total of $430.09 + $207.7 = $<<430.09+207.7=637.79>>637.79 for this week's work.\n#### 637.79 \n question : "+item['question'] +" answer: "+ EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama_1b_test_answers.append(response)'''



  model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

chat_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instruction = "solve the following grade school math problem"
    inputs       = examples["question"]
    outputs      = examples["answer"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = chat_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

# Uncomment for the correct dataset
dataset = load_dataset('json', data_files=str(finetune_1_prime_nos_path) +"/replaced_dataset/train_socratic_output.jsonl", )# dataset = load_dataset("<your_hf_account>/med-qa", split = "train")

dataset = dataset.map(formatting_prompts_func, batched = True,)


count_text = 0
count_question = 0
count_answer = 0

for item in dataset['train']:
  if item['text'] == None:
    count_text = count_text+ 1
  if item['question'] == None:
    count_question +=1
  if item['answer'] == None:
    count_question +=1


if count_text == 0 and count_question == 0 and count_answer == 0:
  pass
else:
  0/0

from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        num_train_epochs = 1
    ),
)

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)


llama3_1b_finetuned_test_socratic_answers = []

for i in tqdm(range(len(qa_socratic_test_data))):

  item = qa_socratic_test_data[i]

  prompt = "solve the following grade school math problem question : "+item['question'] +" answer: "+ EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama3_1b_finetuned_test_socratic_answers.append(response)

  '''from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)


llama_1b_finetuned_test_answers = []


for item in qa_test_data:
  prompt = "solve the following grade school math problem like this question :Eliza's rate per hour for the first 41 hours she works each week is $10.49. She also receives an overtime pay of 3.3 times her regular hourly rate. If Eliza worked for 47 hours this week, how much are her earnings for this week? answer : Eliza is entitled to 47 -41 = <<47-41=6.0>>6.0 hours overtime pay.\nHer hourly rate for the overtime pay is $10.49 x 3.3 = $<<10.49*3.3=34.62>>34.62.\nSo, Eliza will receive $34.62 x 6.0 =$<<34.62*6.0=207.7>>207.7 for overtime pay.\nHer regular weekly earning is $10.49 x 41 = $<<10.49*41=430.09>>430.09.\nThus, Eliza will receive a total of $430.09 + $207.7 = $<<430.09+207.7=637.79>>637.79 for this week's work.\n#### 637.79 \n question : "+item['question'] +" answer: "+ EOS_TOKEN


  inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


  outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  response = tokenizer.batch_decode(outputs)
  llama_1b_finetuned_test_answers.append(response)'''


llama3_1b_socratic_results = {}
llama3_1b_dir = os.path.join(drive_path ,"finetuning" , "llama3_1b")

llama3_1b_socratic_results["test_results"]=llama3_1b_test_socratic_answers
llama3_1b_socratic_results["finetuned_results"] = llama3_1b_finetuned_test_socratic_answers



with open( os.path.join(llama3_1b_dir,"llama3_1b_socratic_results.json"), "w") as outfile:
    json.dump(llama3_1b_socratic_results, outfile)

model.save_pretrained(os.path.join(llama3_1b_dir,"lora_model_socratic_llama3_1b")) # Local saving
tokenizer.save_pretrained(os.path.join(llama3_1b_dir,"lora_model_socratic_llama3_1b"))


# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [ ]:
del llama3_1b_socratic_results
del llama3_1b_finetuned_test_socratic_answers
del llama3_1b_test_socratic_answers
del model
del tokenizer
del inputs
del outputs
del response
del text_streamer
del trainer
del count_text
del count_question
del count_answer
del dataset
